In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 37 not upgraded.
Need to get 9,604 kB of archives.
After this operation, 29.5 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fonts-nanum all 20170925-1 [9,604 kB]
Fetched 9,604 kB in 2s (5,914 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 155222 files and d

In [ ]:
!pip install -U openpyxl
!pip install pyLDAvis

In [ ]:
!pip install konlpy
!pip install apyori
!pip install networkx
!pip install WordCloud
!pip install nltk

In [ ]:
import requests
from pandas import DataFrame
from bs4 import BeautifulSoup
import re
from datetime import datetime
from datetime import timedelta
import os

from konlpy.tag import Hannanum, Okt
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import re
import collections
from collections import Counter
import os
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pandas as pd
from apyori import apriori
import networkx as nx
from google.colab import files

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn import neighbors, preprocessing
from sklearn.svm import SVC
# from sklearn import linear_model, metrics, preprocessing
from sklearn.preprocessing import MinMaxScaler

import matplotlib.pyplot as plt
from sklearn import tree
import numpy as np

import pyLDAvis.sklearn  # sklearn의 ldamodel에 최적화된 라이브러리

nltk.download("punkt")

In [ ]:
os.mkdir('./이재명')
os.mkdir('./윤석열')
os.mkdir('./심상정')
os.mkdir('./안철수')

In [ ]:
query = input('검색 키워드를 입력하세요 : ')
# news_num = int(input('총 필요한 뉴스기사 수를 입력해주세요(숫자만 입력) : '))
news_num = 300
from_date = input('from 날짜를 입력하세요(yyyyMMdd) : ')
to_date = input('to 날짜를 입력하세요(yyyyMMdd) : ')
query = query.replace(' ', '+')

d = timedelta(days=1)
this_date = datetime.strptime(from_date, '%Y%m%d')
date = this_date.strftime('%Y%m%d')

while to_date != date :
  date = this_date.strftime('%Y%m%d')
  news_url = 'https://search.naver.com/search.naver?where=news&query={}&sm=tab_opt&sort=0&photo=0&field=0&pd=3&ds='+date+'&de='+date

  req = requests.get(news_url.format(query))
  soup = BeautifulSoup(req.text, 'html.parser')

  news_dict = {}
  idx = 0
  cur_page = 1

  print()
  print('크롤링 중...')

  finish = False
  while idx < news_num:
  ### 네이버 뉴스 웹페이지 구성이 바뀌어 태그명, class 속성 값 등을 수정함(20210126) ###
      
      table = soup.find('ul',{'class' : 'list_news'})
      li_list = table.find_all('li', {'id': re.compile('sp_nws.*')})
      area_list = [li.find('div', {'class' : 'news_area'}) for li in li_list]
      a_list = [area.find('a', {'class' : 'news_tit'}) for area in area_list]
      
      for n in a_list[:min(len(a_list), news_num-idx)]:
          news_dict[idx] = {'title' : n.get('title'),
                            'url' : n.get('href') }
          idx += 1

      cur_page += 1

      pages = soup.find('div', {'class' : 'sc_page_inner'})
      
      try :
        next_page_url = [p for p in pages.find_all('a') if p.text == str(cur_page)][0].get('href')
      except Exception as e:
        print(e);
        finish = True;
      
      if finish:
        break
      
      req = requests.get('https://search.naver.com/search.naver' + next_page_url)
      soup = BeautifulSoup(req.text, 'html.parser')


  print('크롤링 완료')
  print('데이터프레임 변환')
  news_df = DataFrame(news_dict).T

  folder_path = os.getcwd()
  xlsx_file_name = '네이버뉴스_{}_{}.xlsx'.format(query, date)

  news_df.to_excel(xlsx_file_name)
  print('엑셀 저장 완료 | 경로 : {}\\{}'.format(folder_path, xlsx_file_name))
  this_date = this_date+d

In [ ]:
#불용어
# stop = '대선,후보,대통령,발언,제안,홍준표,유승민,지지율,주자,여론조사,최재형,원희룡,추미애,이준석,이진숙,박정희,안철수,심상정,박용진,이재명' 
# stop = '대선,후보,대통령,발언,제안,홍준표,유승민,지지율,주자,여론조사,최재형,원희룡,윤석열,추미애,이준석,이진숙,박정희,안철수,심상정,박용진'
# stop = '대선,후보,대통령,발언,제안,홍준표,유승민,지지율,주자,여론조사,최재형,원희룡,윤석열,추미애,이준석,이진숙,박정희,심상정,박용진,이재명'
# stop = '대선,후보,대통령,발언,제안,홍준표,유승민,지지율,주자,여론조사,최재형,원희룡,윤석열,추미애,이준석,이진숙,박정희,안철수,박용진,이재명'
stop = '대선,후보,대통령,발언,제안,홍준표,유승민,지지율,주자,여론조사,최재형,원희룡,윤석열,추미애,이준석,이진숙,박정희,심상정,박용진,이낙연,문재인,안철수,이재명,축하,국민,의힘,국민의힘,노무현,민주당'
stop_list = stop.split(',')

In [ ]:
os.chdir('/content/이재명')
os.getcwd()

'/content/이재명'

In [ ]:
xls_list = os.listdir()

for i in xls_list:
  if i.endswith('.xlsx'):
    news_df = pd.read_excel('./'+i)

    okt = Okt()
    transactions = []
    nodesizing = []
    
    for i in news_df.title:
      
      for j in stop_list:                       #불용어 제거
        i = i.replace(j, '')

      tok = word_tokenize(i)
      i = re.sub('[^A-Za-z0-9가-힣\s]', ' ', i) #특수문자 제거
      
      i = okt.nouns(i)
      
      transactions.append(i)
      nodesizing += i

vocab = Counter(nodesizing)
vocab_size = 30
vocab2 = vocab.most_common(vocab_size)

dictionary = dict.fromkeys(vocab2,0)
dic2 = str(dictionary)

In [ ]:
dic2

"{('윤희', 49): 0, ('숙', 46): 0, ('충청', 34): 0, ('대전', 25): 0, ('표심', 19): 0, ('경선', 17): 0, ('사퇴', 17): 0, ('주', 16): 0, ('충남', 15): 0, ('포토', 15): 0, ('공약', 14): 0, ('정치', 13): 0, ('것', 12): 0, ('위', 12): 0, ('부동산', 11): 0, ('잡기', 10): 0, ('첫', 10): 0, ('대권', 10): 0, ('지지', 10): 0, ('뉴스', 10): 0, ('수사', 10): 0, ('방문', 9): 0, ('호남', 9): 0, ('연속', 8): 0, ('주말', 8): 0, ('목소리', 8): 0, ('김어준', 8): 0, ('지사', 8): 0, ('차', 8): 0, ('언론', 8): 0}"

In [ ]:
#Wordcloud
#해당 부분은 오류가 발생할 수 있습니다.
spwords = set(STOPWORDS)
wc1 = WordCloud(max_font_size=200, stopwords=spwords, font_path='./NanumGothic.ttf',
                background_color='white', width=800, height=800)

wc1.generate(dic2)
plt.figure(figsize=(10, 8))
plt.imshow(wc1)
plt.tight_layout(pad=0)
plt.axis('off')
plt.show()

In [ ]:
#한글자 제거
nodesizing_temp = []
transactions_temp = [[],]

for i in nodesizing:
  if len(i) == 1:
    continue
  nodesizing_temp.append(i)

# nodesizing = nodesizing_temp

for i in range(len(transactions)):
  line = []
  for j in range(len(transactions[i])):
    if len(transactions[i][j]) == 1:
      continue
    line.append(transactions[i][j])
  transactions_temp.append(line)

nodesizing = nodesizing_temp
transactions = transactions_temp

In [ ]:
count = Counter(nodesizing)
node_df = pd.DataFrame(count.items(), columns=['node', 'nodesize'])
node_df

,node,nodesize
0,전술핵,3
1,배치,1
2,국힘,1
3,위험천만,1
4,포퓰리즘,4
...,...,...
544,의료,1
545,노조,1
546,총파업,1
547,칼럼,1


In [ ]:
# 연관 분석을 수행합니다.
results = list(apriori(transactions,
    min_support=0.01,
    min_confidence=0.05,
    min_lift=1.0,
    max_length=6))

# 데이터 프레임 형태로 정리합니다.
columns = ['source', 'target', 'support']
network_df = pd.DataFrame(columns=columns)

# 규칙의 조건절을 source, 결과절을 target, 지지도를 support 라는 데이터 프레임의 피처로 변환합니다.
for result in results:
    if len(result.items) == 2:
        items = [x for x in result.items]
        row = [items[0], items[1], result.support]
        series = pd.Series(row, index=network_df.columns)
        network_df = network_df.append(series, ignore_index=True)

network_df 

,source,target,support
0,사과,거짓,0.016529
1,윤희,거짓,0.016529
2,해명,거짓,0.016529
3,공략,경선,0.012397
4,경선,방문,0.020661
...,...,...,...
233,표심,충남,0.016529
234,표심,충청,0.041322
235,테러,카불,0.012397
236,피해자,코스프레,0.016529


In [36]:
node_df.sort_values('nodesize', ascending=False)

,node,nodesize
76,윤희,49
19,충청,34
38,대전,25
32,표심,19
87,경선,17
...,...,...
390,불과,1
46,과반,1
392,의사,1
393,존중,1


In [ ]:
os.getcwd()

'/content/이재명'

In [ ]:
# 그래프 그리기
# 해당 부분은 오류가 발생할 수 있습니다.
plt.rc('font', family='NanumGothic')
plt.figure(figsize=(25,25))

# networkx 그래프 객체를 생성합니다.
G = nx.Graph()

# node_df의 키워드 빈도수를 데이터로 하여, 네트워크 그래프의 ‘노드’ 역할을 하는 원을 생성합니다.
for index, row in node_df.iterrows():
    G.add_node(row['node'], nodesize=row['nodesize'])

# network_df의 연관 분석 데이터를 기반으로, 네트워크 그래프의 ‘관계’ 역할을 하는 선을 생성합니다.
for index, row in network_df.iterrows():
    G.add_weighted_edges_from([(row['source'], row['target'], row['support'])])

# 그래프 디자인과 관련된 파라미터를 설정합니다.
pos = nx.spring_layout(G, k=0.6, iterations=50)
sizes = [G.nodes[node].nodesize]*250 for node in G]
nx.draw(G, pos=pos, node_size=sizes)

# labels = nx.get_edge_attributes(G, 'support')
# nx.draw_networkx_edge_labels(G, pos, edge_labels = labels)
nx.draw_networkx_labels(G, pos=pos, font_family="NanumGothic", font_size=25)

# 그래프를 출력합니다.
ax = plt.gca()

SyntaxError: ignored

In [ ]:
##토픽 모델링
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import konlpy
import re
import os

# tokenization 함수를 만들어둠
def tokenize_korean_text(text):
    text = re.sub(r'[^,.?!\w\s]','', text)  ## ,.?!와 문자+숫자+_(\w)와 공백(\s)만 남김  # 앞에 r을 붙여주면 deprecation warning이 안뜸 (raw string으로 declare)
    
    okt = konlpy.tag.Okt()
    Okt_morphs = okt.pos(text)   # stem=True로 설정하면 동사원형으로 바꿔서 return
    
    words = []
    for word, pos in Okt_morphs:
        if pos == 'Adjective' or pos == 'Verb' or pos == 'Noun':  # 이 경우에는 형용사, 동사, 명사만 남김
            words.append(word)

    words_str = ' '.join(words)
    return words_str


# review_df['Content']를 하나씩 tokenize해서 list로 저장
tokenized_list = []

xls_list = os.listdir()
nodesizing = []

transactions = []
for i in xls_list:
  if i.endswith('.xlsx'):
    news_df = pd.read_excel('./'+i)
    for text in news_df['title']:
        tokenized_list.append(tokenize_korean_text(text))

print(len(tokenized_list))
print(tokenized_list[0])


48728
검찰 황무 사직 강요 의혹 이재명 고발 사건 수사 착수


In [ ]:
#LDA 는 Count기반의 Vectorizer만 적용 
count_vectorizer = CountVectorizer(max_df=0.1, max_features=1000, min_df=2, ngram_range=(1,2))
    # 2개의 문서 미만으로 등장하는 단어는 제외, 전체의 10% 이상으로 자주 등장하는 단어는 제외
    # bigram도 포함

feat_vect = count_vectorizer.fit_transform(tokenized_list)
print('CountVectorizer Shape:', feat_vect.shape)

CountVectorizer Shape: (48728, 1000)


In [ ]:
lda = LatentDirichletAllocation(n_components=6)  # 토픽 수는 6개로 설정
lda.fit(feat_vect)

LatentDirichletAllocation(n_components=6)

In [ ]:
def display_topics(model, feature_names, num_top_words):
    for topic_index, topic in enumerate(model.components_):
        print('Topic #', topic_index)

        # components_ array에서 가장 값이 큰 순으로 정렬했을 때, 그 값의 array index를 반환. 
        topic_word_indexes = topic.argsort()[::-1]
        top_indexes=topic_word_indexes[:num_top_words]
        
        # top_indexes대상인 index별로 feature_names에 해당하는 word feature 추출 후 join으로 concat
        feature_concat = ' '.join([feature_names[i] for i in top_indexes])                
        print(feature_concat)

# CountVectorizer객체내의 전체 word들의 명칭을 get_features_names( )를 통해 추출
feature_names = count_vectorizer.get_feature_names()

# Topic별 가장 연관도가 높은 word를 10개만 추출
display_topics(lda, feature_names, 10)

Topic # 0
국민 홍준표 이재명 윤석열 원금 재난 재난 원금 논란 윤석열 이재명 지지율 해야
Topic # 1
경선 대장동 의혹 이재명 대장동 과반 이재명 이낙연 연기 국감 대장동 의혹 민주당
Topic # 2
기본소득 사퇴 정책 원희룡 대결 공약 기본 국민 이재명 기본소득 청년
Topic # 3
지사 하는 경기도 하는 이재명 이재명 지사 경기 이재명 경기 이재명 경기도 경기 지사 본선
Topic # 4
대선 후보 이재명 이낙연 주자 민주당 이재명 후보 선언 대선 후보 대권 대선 주자
Topic # 5
캠프 대통령 정세균 지지 이재명 캠프 이준석 고발 의혹 의원 이재명 지지


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
import pyLDAvis.sklearn  # sklearn의 ldamodel에 최적화된 라이브러리

pyLDAvis.enable_notebook()
vis = pyLDAvis.sklearn.prepare(lda, feat_vect, count_vectorizer)
pyLDAvis.display(vis)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


In [ ]:
##연관분석 데이터를 사용한 분류 모형 적용
####연관도 순서로 정렬
network_df = network_df.sort_values('support')

###target 이 대선후보인 source를 변수로 설정
columns = network_df.source
col_list = list(set(columns.tolist()))[:20] #중복제거

col_list

['맹비난',
 '슈퍼',
 '재판',
 '윤희',
 '연속',
 '급부',
 '산업',
 '완성',
 '중재법',
 '원샷',
 '작전',
 '해명',
 '군기',
 '운행',
 '충남',
 '아이',
 '달린다',
 '다시',
 '단지',
 '전술핵']

In [ ]:
os.getcwd()

'/content/이재명'

In [ ]:
#일별 빈도수 데이터
results_df = []
xlsx_list = os.listdir()

##########################
for xls_name in xlsx_list:
  
  if xls_name.endswith('.xlsx'):
    df = pd.read_excel('./'+xls_name)

    sentences_temp = []

    for i in df.title:
      i = re.sub('[^A-Za-z0-9가-힣\s]', '', i)
      
      tok = word_tokenize(i)

      sentences_temp += okt.nouns(i)

      line = [0 for i in range(20)]
      # line = [0 for i in range(14)] #안철수 변수 14개

      for i in sentences_temp:
        for j in range(len(col_list)):
          if i == col_list[j]:
            line[j] += 1
      
    results_df.append(line)


In [ ]:
df = pd.DataFrame(results_df)
df.columns = col_list
df.to_csv('/content/이재명.csv')
df

,범주,지시,몸통,광역,이정수,관계,흑백,변호,노영민,소년,센터,결재,사비,한번,대면,만배,대장동,행사,담합,경기
0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3
1,0,2,0,0,0,1,0,2,0,0,0,3,0,0,0,1,44,0,0,3
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,41
3,0,0,4,0,0,0,0,0,0,0,0,2,0,0,0,23,16,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,1
162,0,6,0,0,0,5,0,0,0,0,0,0,0,0,0,0,5,0,0,0
163,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,1,6,1,0,0
164,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [ ]:
# X = pd.read_csv('/content/윤석열.csv', encoding='CP949')
X = pd.read_csv('/content/이재명.csv', encoding='UTF8')
Y = pd.read_csv('/content/google.csv', encoding='CP949')

google_data = X.iloc[:, 2:]
google_data.insert(0,'날짜',Y['일자'])
google_data['관심도'] = Y.iloc[:,2]

In [ ]:
google_data

,날짜,지시,몸통,광역,이정수,관계,흑백,변호,노영민,소년,센터,결재,사비,한번,대면,만배,대장동,행사,담합,경기,관심도
0,2021-06-01,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,5
1,2021-06-02,2,0,0,0,1,0,2,0,0,0,3,0,0,0,1,44,0,0,3,-6
2,2021-06-03,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,41,1
3,2021-06-04,0,4,0,0,0,0,0,0,0,0,2,0,0,0,23,16,0,0,0,-1
4,2021-06-05,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,12,-2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161,2021-11-09,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,1,-1
162,2021-11-10,6,0,0,0,5,0,0,0,0,0,0,0,0,0,0,5,0,0,0,-1
163,2021-11-11,0,0,0,0,2,0,0,0,0,0,0,0,0,0,1,6,1,0,0,-4
164,2021-11-12,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2


In [ ]:
Y = google_data.관심도 > 0
X = google_data.iloc[:, 1:10]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split( X, Y, test_size=0.3)

scaler = preprocessing.MinMaxScaler()
X_train = scaler.fit_transform( X_train)
X_test  = scaler.fit_transform( X_test  )

In [ ]:
#NB
clf = MultinomialNB().fit(X_train, y_train)
pred = clf.predict(X_test)
print("NB : ", np.mean(pred == y_test))

#DT
dt = tree.DecisionTreeClassifier().fit(X_train, y_train)
pred = dt.predict( X_test)
print("DT : ", np.mean( pred == y_test ))

#RF
rf = RandomForestClassifier(n_estimators=100, oob_score=True, random_state=123456).fit(X_train, y_train)
pred = rf.predict(X_test)
print("RF : ", np.mean( pred == y_test ))

#svc
svc = SVC().fit(X_train, y_train)
pred = svc.predict(X_test)
print("SVC : ", np.mean(pred == y_test))

#knn
knn = neighbors.KNeighborsClassifier().fit(X_train, y_train)
pred = knn.predict(X_test)
knn.kneighbors(X_test, return_distance=False)
print("KNN : ", np.mean(pred == y_test))

NB :  0.42
DT :  0.48
RF :  0.44
SVC :  0.42
KNN :  0.4


In [ ]:
from sklearn.datasets import fetch_20newsgroups
# LDA는 빈도수에만 기반하는 CountVectorizer사용함!
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [ ]:
count_vect = CountVectorizer(max_df=0.95, max_features=1000,
                            min_df=2, stop_words='english',
                            ngram_range=(1,2))
ftr_vect = count_vect.fit_transform(nodesizing)

# LDA클래스를 이용해서 피처 벡터화시킨 것을 토픽모델링 시키기
# 8개의 주제만 뽑았으니 n_components(토픽개수) 8로 설정
lda = LatentDirichletAllocation(n_components=8, random_state=42)
lda.fit(ftr_vect)

# components_속성은 8개의 토픽별(row)로 1000개의 feature(단어)들의 분포수치(column)를 보여줌
print(lda.components_.shape)
print(lda.components_)

(8, 28)
[[ 0.12500011  0.12500011  0.12500009  0.12500011  3.12499949  0.12500007
   0.12500012  0.12500011  0.12500026  0.12500011  0.12500006  0.12500011
   5.12499952  0.12500012  0.12500012  0.12500022  0.12500023  0.12500011
   0.12500026  0.12500013  0.1250001   0.12500009  0.12500011  0.12500011
   0.12500009  0.12500026  0.12500009  0.12500011]
 [ 0.12500009  4.12499935  0.12500007  0.12500009  0.12500007  0.12500005
   2.12499928  0.12500009  0.1250002   0.12500009  0.12500005  0.12500009
   0.12500006  2.12499928  0.12500009  0.12500017  0.12500018  0.12500009
   0.1250002   0.1250001   0.12500008  0.12500007  0.12500009  0.12500009
   0.12500007  0.1250002   0.12500007  3.12499933]
 [ 0.12500004  0.12500004  0.12500003  0.12500004  0.12500003  0.12500002
   0.12500004  0.12500004  2.1249983   0.12500004  0.12500002  0.12500004
   0.12500003  0.12500004  0.12500004 16.12499856  6.12499851  0.12500004
   2.1249983   0.12500004  0.12500004  0.12500003  0.12500004  0.12500004
  

In [ ]:
# 이 때 lda_model이란, 벡터화시킨 텍스트 데이터를 fit까지만 적용한 모델!
def display_topic_words(lda_model, feature_names, num_top_words):
    for topic_idx, topic in enumerate(lda_model.components_):
        print('\nTopic #', topic_idx+1)
        
        # Topic별로 1000개의 단어들(features)중에서 높은 값 순으로 정렬 후 index를 반환해줌!
        # argsort()는 디폴트가 오름차순임(1,2,3,...) 그래서 [::-1]로 내림차순으로 바꿔주기
        topic_word_idx = topic.argsort()[::-1]
        top_idx = topic_word_idx[:num_top_words]
        
        # CountVectorizer함수 할당시킨 객체에 get_feature_names()로 벡터화시킨 feature(단어들)볼 수 있음!
        # 이 벡터화시킨 단어들(features)은 숫자-알파벳순으로 정렬되며, 단어들 순서는 fit_transform시키고 난 이후에도 동일!
        # '문자열'.join 함수로 특정 문자열 사이에 끼고 문자열 합쳐줄 수 있음.
        feature_concat = '+'.join([str(feature_names[i])+'*'+str(round(topic[i], 1)) for i in top_idx])
        print(feature_concat)        
feature_names = count_vect.get_feature_names()
display_topic_words(lda, feature_names, 15)


Topic # 1


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


IndexError: ignored